# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление проекта:
* 1. [Изучение общей информации данных](#step1)
    * 1.1. [Вывод](#step2)
* 2. [Предобработка данных](#step3)
    * 2.1. [Работа с пропущенными значениями и изменение типов данных](#step4)
    * 2.2. [Вывод](#step5)
* 3. [Добавление новых столбцов](#step6) 
    * 3.1. [Вывод](#step7)
* 4. [Исследовательский анализ данных](#step8)  
    * 4.1. [Подсчет среднего и медианы времени продажи квартиры](#step9)
    * 4.2. [Анализ зависимости параметров](#step10)
    * 4.3. [Топ 10 с наибольшим числом объявлений](#step11)
    * 4.4. [Анализ квартир в Санкт-Петербурге](#step12)
    * 4.5. [Анализ квартир в центре](#step13)
* 5. [Общий вывод](#step14) 

### Откроем файл с данными и изучите общую информацию.  <a name="step1"></a>

In [447]:
import pandas as pd
df = pd.read_csv("/datasets/data.csv") #Прочитаем данный файл и посмотрим подробную информацию о нём.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод <a name="step2"></a>

Всего в таблице 12 столбцов, типы данных у столбц - float64(2), int64(5), object(5).

## Предобработка данных

### Обработка пропусков

In [448]:
#Проверим стобцы на пропуски.
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружили, что в столбцах days_employed и total_income есть пропущенные значения. Постараемся избавится от них.

In [449]:
#С помощью метода fillna() заменим пропущенные значения на медиану.
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

In [450]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

С помощью метода fillna() заменим пропущенные значения на медиану в столбац total_income и days_employed.
Традиционными причинами, приводящими к появлению пропусков, является невозможность их получения или обработки.

### Замена типа данных

In [451]:
# В столбце days_employed и total_income с помощью astype() изменим вещественный тип на целочисленный, а также избавимся от отрицательных значений методом abs()
df['days_employed'] = df['days_employed'].astype(int).abs()
df['total_income'] = df['total_income'].astype(int).abs()
df #Выведем таблицу и посмотрим что все сработало 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [452]:
#Посмотрим что у нас находится в столбце gender
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Выявили, что есть неизвестное значение XNA, попробуем от него избавится.

In [453]:
df = df.drop(df[df['gender'] == 'XNA'].index)
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

В столбце education (Образование) выявили данные с разным регистром, приведем их в одному регистру для удобства анализа данных.

In [454]:
df['education'] = df['education'].str.lower()
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

В столбце days_employed и total_income с помощью astype() изменим вещественный тип на целочисленный, а также избавимся от отрицательных значений методом abs(). 
Также выявили в столбце gender XNA и удалим это значение. 

### Обработка дубликатов

In [455]:
df.duplicated().sum()

71

Обнаружили дубликаты, которые мешают для корректного анализа данных. Поэтому удалим их.

In [456]:
df = df.drop_duplicates().reset_index(drop = True)

In [457]:
df.duplicated().sum()

0

### Вывод

Дубликаты могли появится вследствие технического сбоя при записи данных. Выявили дубликаты методом duplicated().sum(). В результате получили 54 дубликата, которые мешают корректному анализу данных, поэтому их нужно удалить.
Для этого используем метод drop_duplicates().

### Лемматизация

In [458]:
#Импортируем библиотеку с функцией лематтизации pymystem3 
from pymystem3 import Mystem
m = Mystem()
df['lemmas'] = df.purpose.apply(m.lemmatize)

In [459]:
#Посчитаем сколько раз встречается каждый спектр в столбце цель кредита.
from collections import Counter
Counter(m.lemmatize(' '.join(df['purpose'])))

Counter({'покупка': 5896,
         ' ': 55021,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6350,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

Заметим, что можно сгруппировать данные столбца цель кредита и запишем эти данные в новый столбец.

In [460]:
#Создадим функцию которая будет возвращать обобщенную цель кредита по столбцу purpose. По следующему правилу:
def purpose(text):
    if 'автомобиль' in text:
        return 'автомобиль'
    if 'свадьба' in text:
        return 'свадьба'
    if 'образование' in text:
        return 'образование'
    if 'жилье' or 'недвижимость' in text:
        return 'недвижимость'

df['lemmas'] = df['lemmas'].apply(purpose)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21449,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21450,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21451,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


In [461]:
Counter(m.lemmatize(' '.join(df['lemmas'])))

Counter({'недвижимость': 10810,
         ' ': 21452,
         'автомобиль': 4306,
         'образование': 4013,
         'свадьба': 2324,
         '\n': 1})

### Вывод

Создали новый столбец lemmas, где разбили по категориям столбец с целями кредита. Получили сгруппированные данные. 
Из этого можно сделать вывод, что больше всего кредитов берут на недвижимость. 
Оно и ясно, ставка 6,5%)

### Категоризация данных

In [462]:
#Создадим функцию,которая категотизирует данные по количеству детей
def children_group(children):
    
    if children == 0:
        return "нет детей"
    
    if children == 1:
        return "один ребенок"
    
    if children == 2:
        return "два ребенка"
    
    if children >= 3:
        return "многодетный"
    
df['children_group'] = df['children'].apply(children_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,один ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,один ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,два ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей


In [463]:
#Создадим функцию,которая категотизирует данные по доходу
def total_income_group(total_income):
    
    if total_income < 90000:
        return "малоимущий"
    
    if total_income >= 90000 and total_income <= 150000:
        return "средний доход"
    
    if total_income > 150000:
        return "обеспеченный"
    
df['total_income_group'] = df['total_income'].apply(total_income_group)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,один ребенок,обеспеченный
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,один ребенок,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный,обеспеченный
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,обеспеченный
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,нет детей,обеспеченный
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,нет детей,обеспеченный
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей,средний доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,два ребенка,средний доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей,средний доход


In [464]:
#Создадим функцию,которая категотизирует данные по наличию долга
def debt_group(debt):
    
    if debt == 1:
        return "есть долг"
    
    if debt == 0:
        return "нет долга"
df['debt_group'] = df['debt'].apply(debt_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_group,total_income_group,debt_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,один ребенок,обеспеченный,нет долга
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,один ребенок,средний доход,нет долга
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,средний доход,нет долга
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный,обеспеченный,нет долга
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,обеспеченный,нет долга
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,нет детей,обеспеченный,нет долга
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,нет детей,обеспеченный,нет долга
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей,средний доход,нет долга
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,два ребенка,средний доход,нет долга
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей,средний доход,нет долга


### Вывод

Данные структурированны, избавились от дубликатов, нулевых и отрицательных значений.
Теперь можно сделать свобдную таблицу и анализировать более подробно.


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [465]:
#Создадим таблицу, где выведем какой процент людей имеет задолжность по каждой группе.
children_from_debt = df.groupby('children_group')['debt'].sum() / df.groupby('children_group')['debt'].count() * 100
children_from_debt

children_group
два ребенка     9.454191
многодетный     8.552632
нет детей       7.544358
один ребенок    9.234609
Name: debt, dtype: float64

### Вывод

Из таблицы можно увидеть, сколько есть долг по тем или иным кредитам.
Из анализа, который мы провели, можно сделать вывод:
Зависимость между наличием детей и возвратом кредита в срок есть. Многодетные семьи имеют меньше долгов, 
чем те у кого нет детей или 1-2 ребенка.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [466]:
#Создадим таблицу, где выведем процентное соотношение задолжности от Семейного положения
family_from_debt = df.groupby('family_status')['debt'].sum() / df.groupby('family_status')['debt'].count() * 100
family_from_debt

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.349398
женат / замужем          7.545182
Name: debt, dtype: float64

### Вывод

Из таблицы можно сделать вывод, что Не женатые/не замужние и люди в гражданском браке имеют больше долгов, чем вдовец/вдова и люди в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [467]:
#Создадим сводную таблицу, где выведем в процентном соотношении зависимость долга от  Дохода
income_group_from_debt = df.groupby('total_income_group')['debt'].sum() / df.groupby('total_income_group')['debt'].count() * 100
income_group_from_debt

total_income_group
малоимущий       7.825568
обеспеченный     7.761953
средний доход    8.652568
Name: debt, dtype: float64

### Вывод

Из таблицы можно сделать вывод, что люди со средним доходом имеют больше долгов, чем малоимущие и обеспеченные.

- Как разные цели кредита влияют на его возврат в срок?

In [468]:
#Создадим сводную таблицу, где выведем зависимость в процентном соотношении целей кредита от вазврата его в срок
lemmas_from_debt = df.groupby('lemmas')['debt'].sum() / df.groupby('lemmas')['debt'].count() * 100
lemmas_from_debt

lemmas
автомобиль      9.359034
недвижимость    7.234043
образование     9.220035
свадьба         8.003442
Name: debt, dtype: float64

### Вывод

Из таблицы можно сделать вывод, что кредит который брали на организацию свадьбы возвращается в срок, нежели кредит на недвижимость.

### Шаг 4. Общий вывод

 После анализа наших исходных данных обнаружили, что есть пропущенные и отрицательные значения, с помощью метода  fillna() заменили пропущенные значения на медианное значение.  
 Избавились от отрицательных значений методом abs(). Обычно такая некорректная выгрузка данных связана с технической ошибкой.
 Для удобства анализа данных привели данные с вещественным типом данных к целочисленному методом astype(int).
Далее обнаружили, что можно сгруппировать данные по целям кредита, доходу клиента, колличеству детей, а также по наличию долга у клиента.  
Для этого создали новые столбцы уже с сгрупированными данными. 
После группировки, вывели в процентном соотношении зависимости долга от колличества детей, дохода клиента и от того на какие цели клиент берет кредит.
Семейное положение и колличества детей клиента безусловно играют роль в погашении кредита в срок. Но нужно учесть, что основной фактр зависит от уровня дохода клиента.